In [1]:
# !python3 --version

In [2]:
# !pip install mxnet

In [3]:
# !pip install gluonnlp pandas tqdm

In [4]:
# !pip install sentencepiece==0.1.91

In [5]:
# !pip install transformers==4.8.2

In [6]:
# !pip install torch==1.8.1

In [7]:
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [8]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [9]:
import numpy as np

if not hasattr(np,'bool'):
    np.bool = bool

/tmp/ipykernel_27560/1911866731.py:3: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(np,'bool'):


In [10]:
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

# KoBert 모델과 토크나이저 불러오기
model, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()

using cached model. /home/azureuser/Desktop/kr/.cache/kobert_v1.zip
using cached model. /home/azureuser/Desktop/kr/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/azureuser/Desktop/kr/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/home/azureuser/anaconda3/envs/kobert3/lib/python3.8/site-packages/transformers/modeling_utils.py:1211: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load

In [11]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

# GPU 사용
device = torch.device('cuda:0')

In [12]:
# 입력 데이터셋 토큰화하기
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding 등을 해주는 코드
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair
        )

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self,i):
        return (self.sentences[i] + (self.labels[i], ))
    
    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters
max_len = 64
batch_size = 16
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [14]:
# !pip install pyarrow

In [15]:
# !pip install pyspark

In [16]:
# SparkSession 생성
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder.appName("Spark로 HDFS CSV 로드") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://127.0.0.1:9000") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/15 09:17:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/15 09:17:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [17]:
# data
data = spark.read.csv('hdfs://localhost:9000/hdfs_folder/KorCCViD.csv', encoding='utf-8')

In [18]:
data.show()

+---------------------------------+-----+
|                              _c0|  _c1|
+---------------------------------+-----+
|                       Transcript|Label|
|   사람 전라도 광주 태장 42 고...|    1|
| 서울 중앙지 숫자 인데 개인 정...|    1|
|    여기 끝내 2005 김창호 도용...|    1|
| 웨딩 으로 원래 그대로 근데 그...|    0|
| 답답 지금 선생 본인 래요 자기...|    1|
| 서울 중앙 지검 김진호 수사관 ...|    1|
|여보세요 들어왔 습니까 수고 십...|    1|
| 우리 호텔 인데 불구 너무 더라...|    0|
| 그래 그서 경우 처음 아니 했잖...|    0|
| 우선 사건 연결 현미 때문 검찰...|    1|
|신한은행 경우 어쩌면 지금 마지...|    1|
| 누구 김강호 구요 서울 출신 입...|    1|
| 오래 동차 라고 동시 봐야 거든...|    0|
| 그래도 뭔가 그런 기회 라도 으...|    0|
| 확인 농협 하나 통장 확인 경기...|    1|
| 대로 그래 카운터 아니 무슨 건...|    0|
|여보세요 방법 국민은행 고객 적...|    1|
| 안녕 세요 씨티 캐피탈 씨티 캐...|    1|
| 초등 학교 학년 학년 친한 친구...|    0|
+---------------------------------+-----+
only showing top 20 rows



In [19]:
df = data.toPandas()

In [20]:
df.head()

,_c0,_c1
0,Transcript,Label
1,사람 전라도 광주 태장 42 고요 명동 에서 10 정도 근무 했었 200 보여 그리...,1
2,서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 습니다 혹시 김용진 라...,1
3,여기 끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외...,1
4,웨딩 으로 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치...,0


In [21]:
# index 0 행 제거
df.drop([0], axis=0, inplace=True)

In [22]:
df.head()

,_c0,_c1
1,사람 전라도 광주 태장 42 고요 명동 에서 10 정도 근무 했었 200 보여 그리...,1
2,서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 습니다 혹시 김용진 라...,1
3,여기 끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외...,1
4,웨딩 으로 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치...,0
5,답답 지금 선생 본인 래요 자기 성함 아니 에요 아니 통해서 통화 어요 답답 십니다...,1


In [23]:
# index 초기화
df.reset_index(drop=True,inplace=True)

In [24]:
df.head()

,_c0,_c1
0,사람 전라도 광주 태장 42 고요 명동 에서 10 정도 근무 했었 200 보여 그리...,1
1,서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 습니다 혹시 김용진 라...,1
2,여기 끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외...,1
3,웨딩 으로 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치...,0
4,답답 지금 선생 본인 래요 자기 성함 아니 에요 아니 통해서 통화 어요 답답 십니다...,1


In [25]:
# column명 변경
df.columns = ['text','label']

In [26]:
df.head()

,text,label
0,사람 전라도 광주 태장 42 고요 명동 에서 10 정도 근무 했었 200 보여 그리...,1
1,서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 습니다 혹시 김용진 라...,1
2,여기 끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외...,1
3,웨딩 으로 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치...,0
4,답답 지금 선생 본인 래요 자기 성함 아니 에요 아니 통해서 통화 어요 답답 십니다...,1


In [27]:
# !pip install nltk

In [28]:
import re

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z]'    # 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    return text

In [29]:
# 불필요한 텍스트 제거
for i in range(len(df)):
    df['text'][i] = text_clean(df['text'][i])

In [30]:
# 숫자는 의미가 있다고 판단하여 제거하지 않았다

In [31]:
# 불용어(stopwords) 처리
# 불용어 사전 불러오기
# sw = spark.read.text('hdfs://localhost:9000/hdfs_folder/stopwords-ko.txt')
sw_df = pd.read_csv('/home/azureuser/Desktop/kr/stopwords-ko.txt', names=['value'], encoding='utf-8')

In [32]:
# sw_df = sw.toPandas()

In [33]:
sw_df.head()

,value
0,가
1,가까스로
2,가령
3,각
4,각각


In [34]:
stop_words = []
for i in range(len(sw_df)):
    stop_words.append(sw_df['value'][i])

In [35]:
stop_words

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하자면',
 '대해서',
 '댕그',


In [36]:
# 불용어 제거
for i in range(len(df)):
    text = df['text'][i]
    tokens = text.split(' ')
    tokens = [word for word in tokens if not word in stop_words]
    cleaned_text = ' '.join(tokens)
    df['text'][i] = cleaned_text

In [37]:
df.head()

,text,label
0,사람 전라도 광주 태장 42 고요 명동 10 정도 근무 했었 200 보여 인데 저녁...,1
1,서울 중앙지 숫자 인데 개인 정보 유출 때문 확인 연락 드렸 김용진 라고 으신 처음...,1
2,끝내 2005 김창호 도용 사건 고요 감당 서울 중앙 지검 지능 범죄 수사 외계인 ...,1
3,웨딩 원래 그대로 근데 그것 요즘 웨딩 그런 추세 라고 수원 용인 그러 돌잔치 그렇...,0
4,답답 지금 선생 본인 래요 성함 에요 통해서 통화 어요 답답 십니다 진짜 결과 아실...,1


In [38]:
data_list = []
for ques, label in zip(df['text'], df['label']):
    temp = []
    temp.append(ques)
    temp.append(str(label))

    data_list.append(temp)

In [39]:
# !pip install scikit-learn

In [40]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [41]:
print(tokenizer)

/home/azureuser/Desktop/kr/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [42]:
# tokenizer의 encoding 방식 확인
with open(tokenizer, "rb") as f:
    raw_data = f.read()
    print(raw_data[:100])

b"\n\x0e\n\x05[UNK]\x15\x00\x00\x00\x00\x18\x02\n\x0e\n\x05[PAD]\x15\x00\x00\x00\x00\x18\x04\n\x0e\n\x05[CLS]\x15\x00\x00\x00\x00\x18\x04\n\x0e\n\x05[SEP]\x15\x00\x00\x00\x00\x18\x04\n\x0f\n\x06[MASK]\x15\x00\x00\x00\x00\x18\x04\n\x08\n\x01!\x15+\xde\x03\xc1\n\t\n\x02!'\x15\x93\x05"


In [43]:
# UTF-8이 아닌 binary 형식이기 때문에 UTF-8 형식으로 변환
with open(tokenizer, "r", encoding="latin-1") as f:
    tokens = f.readlines()

with open("tokenizer_vocab.txt", "w", encoding="utf-8") as f:
    f.writelines(tokens)

In [44]:
# tokenizer가 tokenizer 파일 경로 문자열을 반환하여 실제 토크나이저를 로드해야 한다.
from transformers import BertTokenizer

real_tokenizer = BertTokenizer(vocab_file="tokenizer_vocab.txt", do_lower_case=False)

In [45]:
# BERTDataset 클래스를 활용하여 tokenization, int encoding, padding 진행
tok = real_tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [46]:
# torch 형식의 데이터셋 만들어주기
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [47]:
# KoBERT 모델 구현하기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [48]:
#BERT 모델 불러오기
bertmodel = BERTClassifier(model,  dr_rate=0.5).to(device)

In [49]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in bertmodel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in bertmodel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [50]:
#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [51]:
# !pip install -U ipywidgets

In [52]:
# !pip install -U jupyter jupyterlab notebook ipywidgets

In [54]:
# 학습 (Train)
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    bertmodel.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = bertmodel(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(bertmodel.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    bertmodel.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = bertmodel(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/tmp/ipykernel_27560/1102525926.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7217010259628296 train acc 0.625
epoch 1 train acc 0.7701990632318501


/tmp/ipykernel_27560/1102525926.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/16 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8828125


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4244205355644226 train acc 0.875
epoch 5 train acc 0.8953454332552694


  0%|          | 0/16 [00:00<?, ?it/s]

epoch 5 test acc 0.88671875


In [58]:
# 문장을 직접 입력하여 확인
def predict(predict_sentence):

    test_data = [predict_sentence, '0']
    dataset_another = [test_data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)
    
    bertmodel.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = bertmodel(token_ids, valid_length, segment_ids)

        test_eval = []
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("보이스피싱이 아닐 확률이 높습니다. 확률은 ")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱일 확률이 높습니다. 확률은 ")
        
        probs = torch.nn.functional.softmax(out, dim=-1)
        preds = torch.argmax(probs, dim=-1)
        per = round(probs.tolist()[0][preds.item()] * 100,2)

        print(">> " + test_eval[0] + str(per) + "% 입니다.")

In [59]:
sentence = input("")
predict(sentence)

>> 보이스피싱일 확률이 높습니다. 확률은 99.26% 입니다.


In [57]:
'''
본인께서 대가성의 돈을 받고 이 두 통장을 판매하신 계좌 양도 혐의의 가해자이신지, 그게 아니라면 본인이 본인도 모르게 
명의를 도용을 당하고... 자, 그러면 고객님. 고객님께서는 그럼 얼마까지 가능하신데요? 왜 무리라는 거죠? 제가 봤을 때는 
오히려 고객님한테 이득인데? 혹시라도 겁이 나서 은폐 또는 은닉을 하실 시에는 형법 155조 1-1항 증거인멸죄, 형법 152조 위증죄, 
형법 132조 1항 공무집행방해죄가 추가된다는 점 명시를 해드리고...
'''

'\n본인께서 대가성의 돈을 받고 이 두 통장을 판매하신 계좌 양도 혐의의 가해자이신지, 그게 아니라면 본인이 본인도 모르게 \n명의를 도용을 당하고... 자, 그러면 고객님. 고객님께서는 그럼 얼마까지 가능하신데요? 왜 무리라는 거죠? 제가 봤을 때는 \n오히려 고객님한테 이득인데? 혹시라도 겁이 나서 은폐 또는 은닉을 하실 시에는 형법 155조 1-1항 증거인멸죄, 형법 152조 위증죄, \n형법 132조 1항 공무집행방해죄가 추가된다는 점 명시를 해드리고...\n'

In [60]:
# 모델 저장 (pkl 파일)
torch.save(bertmodel, '/home/azureuser/Desktop/kr/kobert.pkl')

In [61]:
# 모델 불러오는 코드
# 저장된 모델 불러오기
# kobert = torch.load("kobert.pkl")
# kobert.eval()